02 파일과 feature 동일, autogluon 사용

**AutoGluon TabularPredictor**
- `time_limit`: 7200초 (2시간)
- `presets`: 'best_quality'
- `num_bag_folds`: 5-Fold CV
- `num_stack_levels`: 1 (Multi-layer Stacking)


In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("AutoGluon Experiment")
print("="*60)

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# 2. 전처리 함수 (기존과 동일)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1–2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # 배아 효율 비율 변수들
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 교호작용 변수들
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']
    
    return df_copy

# 3. 전처리 적용
print("\n전처리 중...")
train_df = preprocess(train_df)
test_df = preprocess(test_df)

print(f"전처리 후 Train shape: {train_df.shape}")
print(f"전처리 후 Test shape: {test_df.shape}")

# 4. ID 제거 (AutoGluon은 자동으로 타겟 찾음)
train_data = train_df.drop(columns=['ID', '배아_이식_미도달'])
test_data = test_df.drop(columns=['ID', '배아_이식_미도달'])

print(f"\nFinal Train shape: {train_data.shape}")
print(f"Target distribution:\n{train_data['임신 성공 여부'].value_counts(normalize=True)}")

# ============================
# 5. AutoGluon 학습
# ============================
print("\n" + "="*60)
print("Starting AutoGluon Training")
print("="*60)
print("\n⏰ 예상 소요 시간: 2시간")
print("💾 메모리 사용량: 4-8GB 예상")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label='임신 성공 여부',
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_models'
)

predictor.fit(
    train_data=train_data,
    time_limit=7200,  # 2시간
    presets='best_quality',
    num_bag_folds=5,
    num_bag_sets=1,
    num_stack_levels=1,
    verbosity=2
)

# ============================
# 6. 결과 확인
# ============================
print("\n" + "="*60)
print("AutoGluon Results")
print("="*60)

# Leaderboard
leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 15) ===")
print(leaderboard.head(15).to_string())

# Best model
best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']
print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

# ============================
# 7. Feature Importance
# ============================
print("\n" + "="*60)
print("Feature Importance")
print("="*60)

try:
    importance = predictor.feature_importance(train_data)
    print("\n=== Top 20 Features ===")
    print(importance.head(20).to_string())
except:
    print("Feature importance not available for ensemble model")

# ============================
# 8. 예측 및 Submission
# ============================
print("\n" + "="*60)
print("Generating Predictions")
print("="*60)

# Test 예측
test_predictions = predictor.predict_proba(test_data)

# 확률값 추출 (클래스 1의 확률)
if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission 생성
submission['probability'] = test_proba
submission.to_csv("../outputs/submission_autogluon(0206).csv", index=False)

print("\n✅ AutoGluon Submission saved!")
print(f"   File: ../outputs/submission_autogluon(0206).csv")

# ============================
# 9. 상세 정보 저장
# ============================
print("\n" + "="*60)
print("Saving Details")
print("="*60)

# Leaderboard 전체 저장
leaderboard.to_csv("../outputs/autogluon_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../outputs/autogluon_leaderboard.csv")

# Feature importance 저장
try:
    importance.to_csv("../outputs/autogluon_feature_importance.csv")
    print("✅ Feature importance saved: ../outputs/autogluon_feature_importance.csv")
except:
    pass

# ============================
# 10. 요약
# ============================
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

summary = f"""
Best Model: {best_model}
CV AUC: {best_score:.6f}

Total Models Trained: {len(leaderboard)}
Ensemble Type: {predictor.model_best}

Top 3 Models:
"""

for i in range(min(3, len(leaderboard))):
    summary += f"  {i+1}. {leaderboard.iloc[i]['model']}: {leaderboard.iloc[i]['score_val']:.6f}\n"

print(summary)

# 요약 파일 저장
with open("../outputs/autogluon_summary.txt", "w") as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/autogluon_summary.txt")
print("\n" + "="*60)
print("AutoGluon Experiment Complete! 🎉")
print("="*60)

/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoGluon Experiment

Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_models/ds_sub_fit/sub_fit_ho.


전처리 후 Train shape: (256351, 86)
전처리 후 Test shape: (90067, 85)

Final Train shape: (256351, 84)
Target distribution:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

Starting AutoGluon Training

⏰ 예상 소요 시간: 2시간
💾 메모리 사용량: 4-8GB 예상

학습 시작...



2026-02-06 11:20:02,405	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 1849 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 5351 seconds.
Starting full fit now with num_stack_levels 0.
Beginning AutoGluon training ... Time limit = 5351s
AutoGluon will save models to "../outputs/autogluon_models"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       4.73 GB / 16.00 GB (29.6%)
Disk Space Avail:   227.67 GB / 460.43 GB (49.4%)
Train Data Rows:    256351
Train Data Columns: 83
Label Column:       임신 성공 여부
Problem Type:


AutoGluon Results


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']
Computing feature importance via permutation shuffling for 77 features using 5000 rows with 5 shuffle sets...



=== Model Leaderboard (Top 15) ===
                      model  score_test  score_val eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     ExtraTrees_r42_BAG_L1    0.958349   0.722289     roc_auc        2.102997       8.557869    59.288020                 2.102997                8.557869          59.288020            1       True         22
1   RandomForestEntr_BAG_L1    0.930159   0.731145     roc_auc        2.477784       8.096630    13.980618                 2.477784                8.096630          13.980618            1       True          6
2   RandomForestGini_BAG_L1    0.928349   0.730036     roc_auc        2.458589       7.909825    13.528742                 2.458589                7.909825          13.528742            1       True          5
3     ExtraTreesEntr_BAG_L1    0.903508   0.732631     roc_auc        2.593283       7.924929    11.344786                 2

	853.67s	= Expected runtime (170.73s per shuffle set)
	544.7s	= Actual runtime (Completed 5 of 5 shuffle sets)



=== Top 20 Features ===
                  importance    stddev       p_value  n  p99_high   p99_low
이식된 배아 수            0.019851  0.004014  1.901256e-04  5  0.028117  0.011586
시술 당시 나이            0.011297  0.003085  6.059486e-04  5  0.017649  0.004945
나이_3구간              0.010647  0.003570  1.313894e-03  5  0.017998  0.003296
배아_이식_비율            0.009838  0.002147  2.555314e-04  5  0.014258  0.005418
이식배아_구간             0.009680  0.003194  1.237507e-03  5  0.016256  0.003103
나이×Day5             0.008002  0.002413  8.827383e-04  5  0.012971  0.003033
배아_저장_비율            0.007536  0.002599  1.459067e-03  5  0.012888  0.002184
배아 이식 경과일           0.007275  0.001796  4.120106e-04  5  0.010974  0.003576
총 생성 배아 수           0.006966  0.002667  2.142094e-03  5  0.012457  0.001475
배아_생성_효율            0.006736  0.001112  8.599298e-05  5  0.009026  0.004446
시술 시기 코드            0.006650  0.000500  3.817704e-06  5  0.007680  0.005620
배아_진행_단계            0.004722  0.001765  1.961776e-03  5  0.0083

In [2]:
print("Submission model:", predictor.model_best)

Submission model: WeightedEnsemble_L2


AUC 기준 WeightedEnsemble_L2 1등 0.740718

------------

<계속해서 상위권에 있는 원본 변수>

이식된 배아 수 (0.0199) 

시술 당시 나이 (0.0113)

배아 이식 경과일 (0.0073)

총 생성 배아 수 (0.0067)

저장된 배아 수

-------------

<파생 변수>
배아_이식_비율 

나이×Day5

배아_저장_비율

배아_생성_효율

